In [2]:
!nvidia-smi

Mon Jun 10 06:49:20 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.146.02             Driver Version: 535.146.02   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2080 ...    Off | 00000000:03:00.0 Off |                  N/A |
| 27%   32C    P8               1W / 250W |      1MiB /  8192MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
import os
# Assign GPU number
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [4]:
import json
import os
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
import numpy as np

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

2024-06-10 06:49:21.359619: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [5]:
def load_json(file_path):
    with open(file_path, 'r') as f:
        try:
            data = json.load(f)
        except json.JSONDecodeError:
            data = []
            f.seek(0)
            for line in f:
                try:
                    data.append(json.loads(line))
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON in line: {line}\nError: {e}")
    return data

In [6]:
train_file_path = 'hin_train.json'

In [7]:
if os.path.exists(train_file_path):
    
    train_data = load_json(train_file_path)

    def print_summary(data, name):
        print(f"{name} Data Loaded:")
        print(f"Number of records: {len(data)}")
        if isinstance(data, list) and len(data) > 0:
            print("First record:", data[0])
        elif isinstance(data, dict):
            print("Keys:", list(data.keys()))

    print_summary(train_data, "Train")

Train Data Loaded:
Number of records: 1299155
First record: {'unique_identifier': 'hin1', 'native word': 'जन्मदिवस', 'english word': 'janamdivas', 'source': 'Dakshina', 'score': None}


In [8]:
keys_to_keep = ['native word', 'english word']

def filter_keys(data, keys_to_keep):
    filtered_data = [{key: item[key] for key in keys_to_keep if key in item} for item in data]
    return filtered_data

# def print_sample(data, name, sample_size=5):
#     print(f"Filtered {name} Data Sample:")
#     for item in data[:sample_size]:
#         print(item)

train_data_filtered = filter_keys(train_data, keys_to_keep)
train = list(train_data_filtered)

# print_sample(train, "Train")

In [9]:
train = pd.DataFrame(train)

train.rename(columns={'native word': 'hindi', 'english word': 'english'}, inplace=True)
train.head()

,hindi,english
0,जन्मदिवस,janamdivas
1,रक्खा,rakha
2,मिलीजुली,milijuli
3,जांचों,jaanchon
4,चमकता,chamkata


In [10]:
train.shape

(1299155, 2)

In [11]:
train['hindi'] = train['hindi'].astype(str)
train['english'] = train['english'].astype(str)

train['hindi'] = train['hindi'].str.strip()
train['english'] = train['english'].str.strip().str.lower()

train = train[~train['english'].str.contains('\.')]
train = train[~train['english'].str.contains(',')]

In [12]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

train = train[train['english'].apply(isEnglish)]

sos = '^'
eos = '$'
train['english'] = sos + train['english'].astype(str) + eos
train['hindi'] = sos + train['hindi'].astype(str) + eos

print(train.head())

        hindi       english
0  ^जन्मदिवस$  ^janamdivas$
1     ^रक्खा$       ^rakha$
2  ^मिलीजुली$    ^milijuli$
3    ^जांचों$    ^jaanchon$
4     ^चमकता$    ^chamkata$


In [13]:
english_text = train['english']
hindi_text = train['hindi']

print('english text shape :', english_text.shape)
print(english_text[0])
print('hindi text shape :', hindi_text.shape)
print(hindi_text[0])

english text shape : (1299155,)
^janamdivas$
hindi text shape : (1299155,)
^जन्मदिवस$


In [14]:
train_english_text, val_english_text, train_hindi_text, val_hindi_text = train_test_split(english_text, hindi_text, test_size=0.2, random_state=42)

In [15]:
# Calculate max input and target lengths
max_input_len = max([len(text) for text in english_text])
print(max_input_len)
max_target_len = max([len(text) for text in hindi_text])
print(max_target_len)

34
30


In [16]:
input_chars = sorted(list(set(''.join(english_text))))
print(input_chars)
target_chars = sorted(list(set(''.join(hindi_text))))
print(target_chars)

num_encoder_tokens = len(input_chars)
print(num_encoder_tokens)
num_decoder_tokens = len(target_chars)
print(num_decoder_tokens)

['$', '^', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
['$', '^', 'ँ', 'ं', 'ः', 'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ', 'ऍ', 'ए', 'ऐ', 'ऑ', 'ओ', 'औ', 'क', 'ख', 'ग', 'घ', 'ङ', 'च', 'छ', 'ज', 'झ', 'ञ', 'ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न', 'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ल', 'ळ', 'व', 'श', 'ष', 'स', 'ह', '़', 'ऽ', 'ा', 'ि', 'ी', 'ु', 'ू', 'ृ', 'ॅ', 'े', 'ै', 'ॉ', 'ॊ', 'ो', 'ौ', '्', 'ॐ', 'ॠ', '॥']
28
71


In [17]:
input_char_index = dict((char, i) for i, char in enumerate(input_chars))
print(input_char_index)
target_char_index = dict((char, i) for i, char in enumerate(target_chars))
print(target_char_index)

{'$': 0, '^': 1, 'a': 2, 'b': 3, 'c': 4, 'd': 5, 'e': 6, 'f': 7, 'g': 8, 'h': 9, 'i': 10, 'j': 11, 'k': 12, 'l': 13, 'm': 14, 'n': 15, 'o': 16, 'p': 17, 'q': 18, 'r': 19, 's': 20, 't': 21, 'u': 22, 'v': 23, 'w': 24, 'x': 25, 'y': 26, 'z': 27}
{'$': 0, '^': 1, 'ँ': 2, 'ं': 3, 'ः': 4, 'अ': 5, 'आ': 6, 'इ': 7, 'ई': 8, 'उ': 9, 'ऊ': 10, 'ऋ': 11, 'ऍ': 12, 'ए': 13, 'ऐ': 14, 'ऑ': 15, 'ओ': 16, 'औ': 17, 'क': 18, 'ख': 19, 'ग': 20, 'घ': 21, 'ङ': 22, 'च': 23, 'छ': 24, 'ज': 25, 'झ': 26, 'ञ': 27, 'ट': 28, 'ठ': 29, 'ड': 30, 'ढ': 31, 'ण': 32, 'त': 33, 'थ': 34, 'द': 35, 'ध': 36, 'न': 37, 'प': 38, 'फ': 39, 'ब': 40, 'भ': 41, 'म': 42, 'य': 43, 'र': 44, 'ल': 45, 'ळ': 46, 'व': 47, 'श': 48, 'ष': 49, 'स': 50, 'ह': 51, '़': 52, 'ऽ': 53, 'ा': 54, 'ि': 55, 'ी': 56, 'ु': 57, 'ू': 58, 'ृ': 59, 'ॅ': 60, 'े': 61, 'ै': 62, 'ॉ': 63, 'ॊ': 64, 'ो': 65, 'ौ': 66, '्': 67, 'ॐ': 68, 'ॠ': 69, '॥': 70}


In [18]:
encoder_input_data = np.zeros((len(train_english_text), max_input_len, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros((len(train_hindi_text), max_target_len, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros((len(train_hindi_text), max_target_len, num_decoder_tokens), dtype='float32')

for i, (input_text, target_text) in enumerate(zip(train_english_text, train_hindi_text)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_char_index[char]] = 1
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_char_index[char]] = 1
        if t > 0:
            decoder_target_data[i, t - 1, target_char_index[char]] = 1

In [19]:
def custom_accuracy(y_true, y_pred):
    y_true_argmax = tf.argmax(y_true, axis=-1)
    y_pred_argmax = tf.argmax(y_pred, axis=-1)
    mask = tf.cast(tf.not_equal(y_true_argmax, 0), dtype=tf.float32)
    matches = tf.cast(tf.equal(y_true_argmax, y_pred_argmax), dtype=tf.float32)
    masked_matches = matches * mask
    accuracy = tf.reduce_sum(masked_matches) / tf.reduce_sum(mask)
    return accuracy

def custom_loss(y_true, y_pred):
    mask = tf.cast(tf.not_equal(tf.reduce_sum(y_true, axis=-1), 0), dtype=tf.float32)
    loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
    masked_loss = loss * mask
    return tf.reduce_sum(masked_loss) / tf.reduce_sum(mask)


In [20]:
model = load_model('D1engHin_model.h5', custom_objects={'custom_loss': custom_loss, 'custom_accuracy':custom_accuracy})


2024-06-10 06:49:43.838900: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2024-06-10 06:49:43.874906: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-06-10 06:49:43.875222: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:0e:00.0 name: NVIDIA GeForce RTX 2080 SUPER computeCapability: 7.5
coreClock: 1.815GHz coreCount: 48 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 462.00GiB/s
2024-06-10 06:49:43.875241: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2024-06-10 06:49:43.877746: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2024-06-10 06:49:43.877778: I tensorflow/stream_execut

In [21]:
# Build the model
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(256, return_state=True, dropout=0.3, recurrent_dropout=0.3)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(256, return_sequences=True, return_state=True, dropout=0.3, recurrent_dropout=0.3)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=custom_loss, metrics=[custom_accuracy])

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 28)]   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 71)]   0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 256), (None, 291840      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  335872      input_2[0][0]                    
                                                                 lstm[0][1]                   

In [22]:
# history = model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
#           batch_size=128,
#           epochs=50,
#           validation_split=0.2)

In [23]:
model.load_weights('D1engHin_model.h5')

In [24]:
val_encoder_input_data = np.zeros((len(val_english_text), max_input_len, num_encoder_tokens), dtype='float32')
val_decoder_input_data = np.zeros((len(val_hindi_text), max_target_len, num_decoder_tokens), dtype='float32')
val_decoder_target_data = np.zeros((len(val_hindi_text), max_target_len, num_decoder_tokens), dtype='float32')

for i, (input_text, target_text) in enumerate(zip(val_english_text, val_hindi_text)):
    for t, char in enumerate(input_text):
        val_encoder_input_data[i, t, input_char_index[char]] = 1
    for t, char in enumerate(target_text):
        val_decoder_input_data[i, t, target_char_index[char]] = 1
        if t > 0:
            val_decoder_target_data[i, t - 1, target_char_index[char]] = 1

In [25]:
scores = model.evaluate([val_encoder_input_data, val_decoder_input_data], val_decoder_target_data, verbose=1)
print('Val loss:', scores[0])
print('Val accuracy:', scores[1])

2024-06-10 06:49:49.571007: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2024-06-10 06:49:49.587571: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 3500030000 Hz
2024-06-10 06:49:50.241101: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11


   7/8120 [..............................] - ETA: 2:51 - loss: 0.2618 - custom_accuracy: 0.8984  

2024-06-10 06:49:50.479522: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11


8120/8120 [==============================] - 178s 22ms/step - loss: 0.2661 - custom_accuracy: 0.9023
Val loss: 0.26612356305122375
Val accuracy: 0.9022973775863647


In [26]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [34]:
import time
def decode_sequence(input_seq):
    start_time = time.time()
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, len(target_chars)))
    target_seq[0, 0, target_char_index['^']] = 1.0
    stop_condition = False
    decoded_sentence = ''
    step = 0
    
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = target_chars[sampled_token_index]
        decoded_sentence += sampled_char
        
        # Debugging statement
        print(f'Step {step}: Sampled char: {sampled_char}, Decoded sentence so far: {decoded_sentence}')
        
        if sampled_char == '$' or len(decoded_sentence) > max_target_len:
            stop_condition = True
            
        target_seq = np.zeros((1, 1, len(target_chars)))
        target_seq[0, 0, sampled_token_index] = 1.0
        states_value = [h, c]
        step += 1
        
    end_time = time.time()
    latency = end_time - start_time
    return decoded_sentence, latency  

In [35]:
def predict_output(input_text):
    total_latency = 0
    words = input_text.split(' ')
    transliterated_words = []
    
    for word in words:
        word = '^' + word.lower() + '$'
        input_seq = np.zeros((1, max_input_len, len(input_chars)), dtype='float32')
        for t, char in enumerate(word):
            if char in input_char_index:
                input_seq[0, t, input_char_index[char]] = 1
        
        print(f'\nInput word: {word}')
        
        decoded_word, latency = decode_sequence(input_seq)
        total_latency += latency
        
        print(f'Decoded word: {decoded_word}')
        
        transliterated_words.append(decoded_word.rstrip('$'))
    
    average_latency = total_latency / len(words) if words else 0
    print(f'Average latency for the input text: {average_latency:.4f} seconds')    
    return ' '.join(transliterated_words)

In [37]:
text = 'Shubhkaamna'
print('English text:',text)
print('Transliterated Hindi text:',predict_output(text))

English text: Shubhkaamna

Input word: ^shubhkaamna$
Step 0: Sampled char: श, Decoded sentence so far: श
Step 1: Sampled char: ु, Decoded sentence so far: शु
Step 2: Sampled char: भ, Decoded sentence so far: शुभ
Step 3: Sampled char: क, Decoded sentence so far: शुभक
Step 4: Sampled char: ा, Decoded sentence so far: शुभका
Step 5: Sampled char: म, Decoded sentence so far: शुभकाम
Step 6: Sampled char: न, Decoded sentence so far: शुभकामन
Step 7: Sampled char: ा, Decoded sentence so far: शुभकामना
Step 8: Sampled char: $, Decoded sentence so far: शुभकामना$
Decoded word: शुभकामना$
Average latency for the input text: 0.3306 seconds
Transliterated Hindi text: शुभकामना


In [38]:
text = 'Rishabh'
print('English text:',text)
print('Transliterated Hindi text:',predict_output(text))

English text: Rishabh

Input word: ^rishabh$
Step 0: Sampled char: ऋ, Decoded sentence so far: ऋ
Step 1: Sampled char: ष, Decoded sentence so far: ऋष
Step 2: Sampled char: भ, Decoded sentence so far: ऋषभ
Step 3: Sampled char: $, Decoded sentence so far: ऋषभ$
Decoded word: ऋषभ$
Average latency for the input text: 0.1664 seconds
Transliterated Hindi text: ऋषभ


In [39]:
text = 'Akshat'
print('English text:',text)
print('Transliterated Hindi text:',predict_output(text))

English text: Akshat

Input word: ^akshat$
Step 0: Sampled char: अ, Decoded sentence so far: अ
Step 1: Sampled char: क, Decoded sentence so far: अक
Step 2: Sampled char: ्, Decoded sentence so far: अक्
Step 3: Sampled char: ष, Decoded sentence so far: अक्ष
Step 4: Sampled char: त, Decoded sentence so far: अक्षत
Step 5: Sampled char: $, Decoded sentence so far: अक्षत$
Decoded word: अक्षत$
Average latency for the input text: 0.2111 seconds
Transliterated Hindi text: अक्षत


In [41]:
text = 'Aaj Hum sab ghumne jayenge'
print('English text:', text)
print('Transliterated Hindi text:', predict_output(text))

English text: Aaj Hum sab ghumne jayenge

Input word: ^aaj$
Step 0: Sampled char: आ, Decoded sentence so far: आ
Step 1: Sampled char: ज, Decoded sentence so far: आज
Step 2: Sampled char: $, Decoded sentence so far: आज$
Decoded word: आज$

Input word: ^hum$
Step 0: Sampled char: ह, Decoded sentence so far: ह
Step 1: Sampled char: म, Decoded sentence so far: हम
Step 2: Sampled char: $, Decoded sentence so far: हम$
Decoded word: हम$

Input word: ^sab$
Step 0: Sampled char: स, Decoded sentence so far: स
Step 1: Sampled char: ब, Decoded sentence so far: सब
Step 2: Sampled char: $, Decoded sentence so far: सब$
Decoded word: सब$

Input word: ^ghumne$
Step 0: Sampled char: घ, Decoded sentence so far: घ
Step 1: Sampled char: ु, Decoded sentence so far: घु
Step 2: Sampled char: म, Decoded sentence so far: घुम
Step 3: Sampled char: न, Decoded sentence so far: घुमन
Step 4: Sampled char: े, Decoded sentence so far: घुमने
Step 5: Sampled char: $, Decoded sentence so far: घुमने$
Decoded word: घुमने$

